In [2]:
import numpy as np

# Foraging ant on an anthill
This notebook simulates a two-dimensional random walk with various boundary conditions(where the food is placed) to find out the average time it would take an ant to reach the food. The problem is challenge by Optiver to be submitted by candidates applying for quantitative researcher roles. The first two parts of the challenge can be done analytically but we will also confirm our results numerically.  
>An ant leaves its anthill in order to forage for food. It moves with the speed of 10 cm per second, but it doesn't know where to go, therefore every second it moves randomly 10 cm directly north, south, east or west with equal probability.

We start by defining a class: `random_walk` upon which we will build our different scenarios. To define this class we need to import the following methods

We need a bunch of ants at some starting (x,y) position which move randomly at the speed of v0 in each time step dt

In [105]:
def forager(ants,steps, sx, sy,dt,v0):
    positions = np.zeros(shape=(ants, 2))
    positions[:, 0] = sx
    positions[:, 1] = sy
    time=np.zeros(shape=(ants))
    #Now implement the random walk
    for i in range(steps):
        # Generate an array with random integers between 0 - 3 to decide the direction of walk
        direction = np.random.randint(low=0, high=4, size=ants)
        
        north = np.where(direction == 0)[0]
        positions[north, 1] += dt * v0

        south = np.where(direction == 1)[0]
        positions[south, 1] -= dt * v0

        east = np.where(direction == 2)[0]
        positions[east, 0] += dt * v0

        west = np.where(direction == 3)[0]
        positions[west, 0] -= dt * v0
        #update the time
        time[~np.isnan(positions[:, 0])] += 1
        # Set a flag to see if ants have reached the boundary
        bdy=boundary(positions)
        positions[bdy, :] = np.nan
    got_food=np.isnan(positions[:,0])
    return time[got_food].mean()

Now we use our code to compute the average time for the first case:

>If the food is located on east-west lines 20 cm to the north and 20 cm to the south, as well as on north-south lines 20 cm to the east and 20 cm to the west from the anthill, how long will it take the ant to reach it on average?


In [106]:
#Now define boundary depending on different cases
#for the first case it is given by
def boundary(pos):
    return(np.abs(pos[:, 0]) >= 20) | (np.abs(pos[:, 1]) >= 20)
print("average time is {:.2f} seconds".format(forager(100000,1000, 0, 0,1,10)))

average time is 4.50 seconds


>What is the average time the ant will reach food if it is located only on a diagonal line passing through (10cm, 0cm) and (0cm, 10cm) points?

In [107]:
def boundary(pos):
    return np.nansum(pos, axis=1) == 10
av_times=[]
for st in range(1,5):
    av_times.append(forager(100000,10**st, 0, 0,1,10))
print(av_times)

[2.272963203262582, 7.613954600800139, 24.520203731657475, 77.91519203041028]


As we can see from above, as we give ants more time, the average time to reach the food does not converge to a single value. This is because the average time is actually infinite due to an open boundary.


> Can you write a program that comes up with an estimate of average time to find food for any closed boundary around the anthill? 

Of course.

>What would be the answer if food is located outside an defined by ( (x – 2.5cm) / 30cm )2 + ( (y – 2.5cm) / 40cm )2 < 1 in coordinate system where the anthill is located at (x = 0cm, y = 0cm)? Provide us with a solution rounded to the nearest integer.


In [108]:
def boundary(pos):
    return np.square((pos[:, 0] - 2.5) / 30) + np.square((pos[:, 1] - 2.5) / 40) >= 1
print("average time is {:.2f} seconds".format(forager(100000,1000, 0, 0,1,10)))

average time is 14.04 seconds
